# Segmentation QC

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
# macOS requirement
import os
import pandas as pd
import geopandas as gpd
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')

In [37]:
import celldega as dega
dataset_metrics_df = dega.qc.ist_segmentation_metrics(transcript_metadata_file="../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_partitioned_transcripts_metadata.parquet",
                                 transcript_data_file="../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-create_subset_subset_coordinates.csv", 
                                 cell_polygon_metadata_file="../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_cell_metadata.parquet",
                                 cell_polygon_data_file="../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_cell_polygons.parquet", 
                                 image_files=["../../../Documents/cell_segmentation/segmentation_data/original_data/Xenium_Prime_Human_Prostate_FFPE_outs/morphology_focus/morphology_focus_0000.ome.tif",
                                              "../../../Documents/cell_segmentation/segmentation_data/original_data/Xenium_Prime_Human_Prostate_FFPE_outs/morphology_focus/morphology_focus_0001.ome.tif",
                                              "../../../Documents/cell_segmentation/segmentation_data/original_data/Xenium_Prime_Human_Prostate_FFPE_outs/morphology_focus/morphology_focus_0002.ome.tif",
                                              "../../../Documents/cell_segmentation/segmentation_data/original_data/Xenium_Prime_Human_Prostate_FFPE_outs/morphology_focus/morphology_focus_0003.ome.tif"], 
                                 subset_interval_y_x=[0,20294,0,42748],
                                 pixel_size=0.2125,
                                 thickness=1)

segmentation metrics calculation completed


In [38]:
dataset_metrics_df

,0_indexed_image_channel_intensity,1_indexed_image_channel_intensity,2_indexed_image_channel_intensity,3_indexed_image_channel_intensity,proportion_transcripts_assigned_to_cells,total_number_of_cells,average_cell_area,average_cell_volume,average_transcripts_per_cell,median_transcripts_per_cell,average_genes_per_cell,median_genes_per_cell,cells_per_100_um^2,percent_empty_cells
0,0.069536,0.154016,0.025722,0.043835,91.175765,222902,2326.244272,2326.244272,244.308589,175.0,179.442647,143.0,0.569,5.702634


In [16]:
trx_meta = pd.read_parquet("../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_partitioned_transcripts_metadata.parquet")
trx = pd.read_csv("../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-create_subset_subset_coordinates.csv")

In [17]:
cell_meta = gpd.read_parquet("../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_cell_metadata.parquet")
cell = gpd.read_parquet("../../../Downloads/data_for_testing_metrics_script/submissions_b41b6b90-1a22-4e63-9fa8-6a80bbeacfc9_MAIN_WORKFLOW_8d99fba5-76b7-466f-afa0-d33105c75de0_call-partitioning_transcript_cell_by_gene_cell_polygons.parquet")